# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [4]:
%pip install -q findspark pyspark

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Start a Local Cluster

In [5]:
import findspark
findspark.init()

In [6]:
spark_url = 'local'

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

24/04/04 20:24:11 WARN Utils: Your hostname, Premikas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.104 instead (on interface en0)
24/04/04 20:24:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 20:24:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [8]:
path = 'netflix-rotten-tomatoes-metacritic-imdb.csv'

In [12]:
df = spark.read.option("delimiter", ",").option("header", True).csv(path)

In [33]:
df.show(20)

+--------------------+--------------------+--------------------+--------------------+---------------+----------------+--------------------+------------+--------------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|               Title|               Genre|                Tags|           Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|            Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For|   Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|           

In [14]:
df.columns


['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series or Movie',
 'Hidden Gem Score',
 'Country Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View Rating',
 'IMDb Score',
 'Rotten Tomatoes Score',
 'Metacritic Score',
 'Awards Received',
 'Awards Nominated For',
 'Boxoffice',
 'Release Date',
 'Netflix Release Date',
 'Production House',
 'Netflix Link',
 'IMDb Link',
 'Summary',
 'IMDb Votes',
 'Image',
 'Poster',
 'TMDb Trailer',
 'Trailer Site']

In [15]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Series or Movie: string (nullable = true)
 |-- Hidden Gem Score: string (nullable = true)
 |-- Country Availability: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- View Rating: string (nullable = true)
 |-- IMDb Score: string (nullable = true)
 |-- Rotten Tomatoes Score: string (nullable = true)
 |-- Metacritic Score: string (nullable = true)
 |-- Awards Received: string (nullable = true)
 |-- Awards Nominated For: string (nullable = true)
 |-- Boxoffice: string (nullable = true)
 |-- Release Date: string (nullable = true)
 |-- Netflix Release Date: string (nullable = true)
 |-- Production House: string (nullable = true)
 |-- Netflix Link: string (nullable = true)
 |-- IMDb Link: string (null

## What is the maximum and average of the overall hidden gem score?

In [19]:
from pyspark.sql.functions import avg, min, max, countDistinct


In [20]:
df.select(avg('Hidden Gem Score'),max('Hidden Gem Score')).show()

+---------------------+---------------------+
|avg(Hidden Gem Score)|max(Hidden Gem Score)|
+---------------------+---------------------+
|    5.937551386501226|                  9.8|
+---------------------+---------------------+



## How many movies that are available in Korea?

In [39]:
df.filter(df.Languages.contains('Korean')).count()

735

## Which director has the highest average hidden gem score?

In [41]:
df.groupBy('Director').agg(avg('Hidden Gem Score').alias('avg_hidden_gem_score')).orderBy('avg_hidden_gem_score', ascending=False).show(1)

+-----------+--------------------+
|   Director|avg_hidden_gem_score|
+-----------+--------------------+
|Dorin Marcu|                 9.8|
+-----------+--------------------+
only showing top 1 row



## How many genres are there in the dataset?

In [48]:
from pyspark.sql.functions import split, explode

df.select(explode(split(df.Genre, ", ")) \
    .alias("genre")) \
    .select("genre") \
    .distinct() \
    .count()


28